In [16]:
import numpy as np
import cv2
import sys

def _thinningIteration2(im, iter):
    I, M = im, np.zeros_like(im)
    rows=I.shape[1]-2
    cols=I.shape[0]-2
    for i in range(1,rows):
        for j in range(1,cols):
            p2 = int(I[i-1][ j ])
            p3 = int(I[i-1][ j+1 ])
            p4 = int(I[i ][j+1 ])
            p5 = int(I[i+1][ j+1 ])
            p6 = int(I[i+1][ j ])
            p7 = int(I[i+1][ j-1 ])
            p8 = int(I[i ][j-1 ])
            p9 = int(I[i-1][ j-1 ])
            #print(p2)
            C  = (1-p2 & (p3 | p4)) + (1-p4 & (p5 | p6)) +\
                     (1-p6 & (p7 | p8)) + (1-p8 & (p9 | p2))
            N1 = (p9 | p2) + (p3 | p4) + (p5 | p6) + (p7 | p8)
            N2 = (p2 | p3) + (p4 | p5) + (p6 | p7) + (p8 | p9)
            N  = N1 if N1 < N2 else N2
            m  = ((p6 | p7 | 1-p9) & p8) if iter == 0 else ((p2 | p3 | 1-p5) & p4)
            if C == 1 and (N >= 2 and N <= 3) and m == 0:
                M[i][j] = 1.
    
    
    return cv2.bitwise_and(I,1.-M)


def thinning(src):
    dst = src.copy() / 255
    prev = np.zeros(src.shape[:2], np.uint8)
    diff = None

    while True:
        dst = _thinningIteration2(dst, 0)
        dst = _thinningIteration2(dst, 1)
        diff = np.absolute(dst - prev)
        prev = dst.copy()
        if np.sum(diff) == 0:
            break

    return dst * 255



src = cv2.imread('C:\\Users\\egor\\Documents\\marking\\ballDir\\98_1.jpg',0)


_, bw2 = cv2.threshold(src, 10, 255, cv2.THRESH_BINARY)
bw2 = thinning(bw2)
res1 = cv2.resize(src,None,fx=5, fy=5, interpolation = cv2.INTER_LINEAR)
cv2.imshow("src", res1)
res = cv2.resize(bw2,None,fx=5, fy=5, interpolation = cv2.INTER_LINEAR)
cv2.imshow("thinning", res)
cv2.imwrite("C:\\Users\\egor\\Documents\\marking\\tem1p.jpg",bw2)
cv2.waitKey()

-1